In [1]:
import torch
torch.cuda.empty_cache()
import torch.nn as nn
from torch import optim
from torch.utils import data
import torch.nn.functional as F
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import string
import re
import random
from torch import Tensor
from torch.nn.init import xavier_uniform_
import math, copy, time
from torch.autograd import Variable
from nltk.tokenize import word_tokenize
from unicodedata import normalize
import numpy as np
from numpy import array

1.8.1+cu102
cuda


In [2]:
from indicnlp.tokenize import indic_tokenize 

# from indicnlp.normalize.indic_normalize import DevanagariNormalizer
# factory = DevanagariNormalizer()

from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer("hi",remove_nuktas=False)

In [3]:
#Lowercase, trim, and remove non-letter characters
def normalizeString(w):
  w = re.sub(r"([?.!,¿-])", r" \1 ", w)
  w = re.sub(r"[().!,?-]+", r" ", w)
  w = re.sub(r'[" "]+', " ", w) #multiple spaces
  w = w.lower().strip() #lowercase #Remove white spaces
  return w

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/shruti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
abbr = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"here's": "here is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"y'all":"you all",
"let's": "let us"
}

In [6]:
import csv
data = []
with open('train2.csv') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=',')
    for row in csvReader:
        data.append([normalizeString(row[1]), normalizeString(row[2])])

In [7]:
for l in data:
  for w in l[1].split():
    if(w in abbr.keys()):
      #print(w)
      l[1] = l[1].replace(w, abbr[w])
      break
data = data[1:]

In [8]:
test = []
with open('hindistatements-3.csv') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=',')
    for row in csvReader:
        test.append([row[2]])

In [9]:
test = test[1:]
len(test)

5000

In [10]:
#Lang class for each language

#So that each sentence has definite starting and ending and model knows where to stop i.e. after EOS. 
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        #map each word to numeric representation
        self.word2index = {"UNK":0, "SOS":1,"EOS":2} #blank is added after EOS to make all sentences of equal length
        self.word2count = {}
        #convert index back to word
        self.index2word = {0:"UNK", 1: "SOS", 2: "EOS"}
        self.n_words = 3  # Count SOS and EOS
    
    #to loop through dataset. 
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    #obtain each word in sentence & update dictionaries
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [11]:
#tokenisation and normalisation
def clean_pairs(lines):
  cleaned = list()
  regex = re.compile('[%s]' % re.escape(string.punctuation))

  for pair in lines:
    clean_pair = list()
    
    #cleaning hindi phrases
    line = pair[0]
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line): 
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = [regex.sub('', w) for w in line]
    #Replace the english characters
    line = [re.sub(r"[a-zA-Z]+?\s", "", w) for w in line]
    line = [re.sub(r'[-.।|,?;:<>&$₹]+','',w) for w in line]
    clean_pair.append(' '.join(line))

    #cleaning english phrases
    line = pair[1]
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = word_tokenize(line) 
    line = [regex.sub('', w) for w in line]
    line = [re.sub(r"[^a-zA-Z0-9?.!,¿\-\/]+", " ", w) for w in line]
    line = [re.sub(r" - ", "-", w) for w in line]
    line = [word for word in line if word.isalpha()]
    clean_pair.append(' '.join(line))
    
    cleaned.append(clean_pair)
  return array(cleaned)

In [12]:
def readLangs(lang1, lang2):
  #make Lang instances
  input_lang = Lang(lang1)
  output_lang = Lang(lang2)

  return input_lang, output_lang

In [13]:
MAX_LENGTH = 60

def filterPair(p):
  return len(p[0].split(' ')) <= MAX_LENGTH and len(p[1].split(' ')) <= MAX_LENGTH

def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

In [14]:
def prepareData(lang1, lang2):
  input_lang, output_lang = readLangs(lang1, lang2)
  print("Read %s sentence pairs" % len(data))
  pairs = clean_pairs(data)
  pairs = filterPairs(pairs)
  print("Trimmed to %s sentence pairs" % len(pairs))
  
  #Make word lists from sentences in pairs. Adding to word vocab
  for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
    
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  return input_lang, output_lang, pairs, test

In [15]:
input_lang, output_lang, tr_pairs, pairst = prepareData('hindi', 'english')
print(random.choice(tr_pairs))

Read 102322 sentence pairs
Trimmed to 101921 sentence pairs
hindi 43660
english 29523
['खैर कुछ अच्छी बात ही बाहर आयी क्योंकि उस ने कहाः'
 'anyway something good did come out of it all because he said']


In [16]:
#tokenization for input lang
def tokenize(lang,sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

#tokenization for output lang
def tokenize2(lang,sentence):
    sent = "SOS" + " " + sentence + " " + "EOS"
    return [lang.word2index[word] for word in sent.split(' ')]

In [17]:
pairs2 = np.array(tr_pairs) #convert the pairs to numpy array

#Run tokenize function on pairs
input_token = list(map(lambda x: tokenize(input_lang,x),pairs2[:,0]))    #all hindi tokens
output_token = list(map(lambda x: tokenize(output_lang,x),pairs2[:,1])) #all english tokens

In [18]:
(pairs2[:,1])

array(['in el salvador both sides that withdrew from their civil war took moves that had been proven to mirror a prisoner s dilemma strategy',
       'i have nothing to do with them', 'fuck them rick', ...,
       'and you know my mother taught us', 'since i was a boy',
       'where did you get something that big'], dtype='<U2007')

In [19]:
#padding sequences to same length

#create zero arrays that will be filled with tokens
input_tokenPad = np.zeros((len(input_token),MAX_LENGTH))
output_tokenPad = np.zeros((len(output_token),MAX_LENGTH))

#loop through tokens and store token at corr index in zero matrix
for i,v in enumerate(input_token):
    for j, token in enumerate(v):
        input_tokenPad[i,j] = token
        
for i,v in enumerate(output_token):
    for j, token in enumerate(v):
        output_tokenPad[i,j] = token

In [20]:
(input_tokenPad.shape)

(101921, 60)

In [21]:
from sklearn.model_selection import train_test_split

train_hin, valid_hin,train_eng,valid_eng = train_test_split(input_tokenPad,output_tokenPad,test_size=0.1,shuffle=True)

In [22]:
from torch.utils.data import TensorDataset, DataLoader

#convert data to tensordataset and make sure sequence is not in float
train_data = TensorDataset(torch.from_numpy(train_hin).long(),torch.from_numpy(train_eng).long())
valid_data = TensorDataset(torch.from_numpy(valid_hin).long(),torch.from_numpy(valid_eng).long())

#create dataloader with batch size
batch_size = 64
train_loader= DataLoader(train_data,shuffle=True,batch_size=batch_size,)
valid_loader =DataLoader(valid_data,shuffle=True,batch_size=batch_size,)

In [23]:
len(train_loader)

1434

In [24]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print("hindi", sample_x)
print(sample_x.size())

hindi tensor([[   55,  2010,    90,  ...,     0,     0,     0],
        [   95,  1831,    12,  ...,     0,     0,     0],
        [  146, 12027,    63,  ...,     0,     0,     0],
        ...,
        [11933, 29825,  2737,  ...,     0,     0,     0],
        [27198,  7688,    12,  ...,     0,     0,     0],
        [   64,  7136,  1488,  ...,     0,     0,     0]])
torch.Size([64, 60])


In [25]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        #encodes symbol rep x to continuous rep z
        self.encoder = encoder
        #decodes z to outputs y
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        #Take in and process masked src and target sequences
        return self.decode(self.encode(src, src_mask), src_mask,tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [26]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [27]:
#encoder/decoder has N=6 identical layers
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [28]:
#batch normalization
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [29]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [30]:
#residual connections around each of the sub-layers followed by layer normalization.
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        #add to output's sublayer the input that entered this sublayer
        return x + self.dropout(sublayer(self.norm(x)))

In [31]:
#each encoder has 2 parts
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        #1) multi-head self-attention mechanism  2) position-wise fully connected FFN
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        #embedding dimension
        self.size = size

    def forward(self, x, mask):
        #Q,K,V from input embedding vector
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [32]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [33]:
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        #embedding dim
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        #Each layer has 3 sub-layers: 1) masked multi-head self-attn mechanism 2) position-wise fc FFN
        #3) multi-head attention over the output of the encoder stack (source attn)
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        #Q,K,V from target embedding vector
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        #using masking to prevent exposing posterior inf from deco. W/o this deco will not learn anything
        #Q from target seq, K & V from input seq
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [34]:
#Compute 'Scaled Dot Product Attention'
def attention(Q, K, V, mask=None, dropout=None):
    #dimension of K and Q - divide by it to make sure that the mean is close to 1 and the standard deviation is close to 0. prevent explosing of product
    d_k = Q.size(-1)
    #print(d_k) - 64
    #multiply query of scoring word and key of the other words & normalize the scores
    score = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
    #to prevent softmax scores on padded "UNK", mask them out
    if mask is not None:
        #replace 0s with -1e9 & not NaN
        score = score.masked_fill(mask == 0, -1e9)
    #apply softmax to get attention weight. sum will be 1. Get relevance score of each word relative to other
    attn_wt = F.softmax(score, dim = -1)
    if dropout is not None:
        attn_wt = dropout(attn_wt)
    #return summary state of each word and softmax scores multiplied by Value
    mul = torch.matmul(attn_wt, V)
    return mul, attn_wt

In [35]:
#dont want to see all words at same time so Mask out subsequent positions
def subsequent_mask(size):
    attn_shape = (1, size, size)
    #expose target words to model one at a time
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    #integer sequence so mask of True/False
    return torch.from_numpy(subsequent_mask) == 0

In [36]:
#scale dot attention at multiple projections = head h=8 parallel attention layers
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_embed, dropout=0.1):
        #Take in model size and number of heads
        super(MultiHeadedAttention, self).__init__()
        assert d_embed % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_embed // h
        self.h = h
        self.linears = clones(nn.Linear(d_embed, d_embed), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            #Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        #create multiple states
        #Do all the linear projections in batch from d_embed => h x d_k 
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linears, (query, key, value))]
        
        #Apply scale dot attention on all the projected vectors in batch independently
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        
        #"Concat" output of each state using a view ; sum up the weighted value vectors
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        #apply a final linear layer
        return self.linears[-1](x)

In [37]:
class Embeddings(nn.Module):
    def __init__(self, d_embed, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_embed)
        self.d_embed = d_embed

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_embed)

In [38]:
#fully connected layers with RELU
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [39]:
#both enco-deco add PE to the embed vector  
class PositionalEncoding(nn.Module):
    def __init__(self, d_embed, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        #Compute the positional encodings in log space
        pe = torch.zeros(max_len, d_embed)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_embed, 2) * -(math.log(10000.0) / d_embed))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        #returns index that shows the precise word’s location in the sentence based on sine and cosine functions
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

In [40]:
def make_model(src_vocab, tgt_vocab, N=6, d_embed=512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_embed)
    ff = PositionwiseFeedForward(d_embed, d_ff, dropout)
    #index that shows the precise word’s location in the sentence based on sine and cosine functions
    position = PositionalEncoding(d_embed, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_embed, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_embed, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_embed, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_embed, tgt_vocab), c(position)),
        Generator(d_embed, tgt_vocab))
    
    #Initialize parameters
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [41]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        #subsequent mask so that model can leverage self-attention
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [42]:
def run_epoch(data_iter, model, loss_compute):
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, data in enumerate(data_iter):
        src, trg = data
        batch = Batch(src.cuda(),trg.cuda())
        out = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 1000 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f" % (i, loss / batch.ntokens))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

In [43]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)) 
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.zero_grad()
        return loss.item()

In [44]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        size = ys.size(0)
        out = model.decode(memory, src_mask,Variable(ys),Variable(subsequent_mask(ys.size(1)).type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

In [45]:
#pad_idx = output_lang.word2index["UNK"]
model = make_model(input_lang.n_words,output_lang.n_words,N=6)
model.cuda()

criterion = nn.CrossEntropyLoss()
criterion.cuda()

/home/shruti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


CrossEntropyLoss()

In [46]:
import time

model_opt = torch.optim.Adam(model.parameters(), lr=0.003, betas=(0.9, 0.98), eps=1e-9)

for epoch in range(20):
  model.train()
  run_epoch(train_loader,model,SimpleLossCompute(model.generator,criterion,model_opt))
  torch.save(model.state_dict(),'transformer1.pt')
  model.eval()
  for i, data in enumerate(valid_loader):
    src,trg = data
    batch = Batch(src.cuda(),trg.cuda())
    src = batch.src[:1]
    src_mask = (src != input_lang.word2index["UNK"]).unsqueeze(-2)
    out = greedy_decode(model.cuda(), src, src_mask, max_len=60, start_symbol=output_lang.word2index["SOS"])
    for i in range(0, src.size(1)):
        sym = input_lang.index2word[src[0, i].item()]
        if sym == "UNK": break
        print(sym, end =" ")
    print()
    print("Translation:", end="\t")
    for i in range(1, out.size(1)):
        sym = output_lang.index2word[out[0, i].item()]
        if sym == "EOS":
            break
        if(sym != "UNK"):
            print(sym, end =" ")
    print()
    print("Target:", end="\t")
    for i in range(1, batch.trg.size(1)):
        sym = output_lang.index2word[batch.trg.data[0,i].item()]
        if sym == "EOS":
            break
    print()
    break
  print(run_epoch(valid_loader,model,SimpleLossCompute(model.generator,criterion,None)))

RuntimeError: CUDA out of memory. Tried to allocate 426.00 MiB (GPU 0; 10.76 GiB total capacity; 5.86 GiB already allocated; 222.44 MiB free; 6.30 GiB reserved in total by PyTorch)

In [ ]:
#torch.save(model.state_dict(),'transformer.pt')

f = open("answer1.txt", "w")

model.eval()
model.cuda()

for i in range(len(test)):
    sent = str(test[i]).split()
    src = torch.LongTensor([[input_lang.word2index[w] for w in sent if w in input_lang.word2index]])
    src = Variable(src)
    src=src.cuda()
    src_mask = (src != input_lang.word2index["UNK"]).unsqueeze(-2)
    out = greedy_decode(model, src, src_mask, max_len=50, start_symbol=output_lang.word2index["SOS"])
    trans = ""
    for i in range(1, out.size(1)):
        sym = output_lang.index2word[out[0, i].item()]
        if sym == "EOS":
            break
        if(sym!="UNK"):
            trans += sym + " "
    trans+="\n"
    #print(trans)
    f.writelines(trans)

f.close()
